In [30]:
print("OK!")

OK!


In [31]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers


In [32]:
PINECONE_API_KEY = "8e9658d0-c1cb-48ce-a2ae-2b136b8bf0b0"
#PINECONE_API_ENV = "gcp-starter"

In [33]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [34]:
extracted_data = load_pdf("data/")

In [35]:
# extracted_data

In [36]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [37]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 851


In [38]:
# text_chunks

In [39]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [40]:
embeddings = download_hugging_face_embeddings()

In [41]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [42]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [43]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

Note: you may need to restart the kernel to use updated packages.


In [44]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key='8e9658d0-c1cb-48ce-a2ae-2b136b8bf0b0')

In [45]:
index_name = "naritwa"

index = pc.Index(index_name)


In [46]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 851}, 'real': {'vector_count': 322}},
 'total_vector_count': 1173}

In [47]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

Note: you may need to restart the kernel to use updated packages.


In [48]:
!pip install openai -q

In [49]:
# OPENAI_API_KEY='sk-proj-nqRs9vvwyOvdfVqSnYXUT3BlbkFJtQbVBwMv3FROgN2JdtEK'

In [50]:
# from langchain_community.document_loaders import TextLoader
# from langchain_openai import OpenAIEmbeddings
# from langchain_text_splitters import CharacterTextSplitter

# loader = TextLoader("data/file.txt")
# documents = loader.load()
# text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
# docs = text_splitter.split_documents(documents)

# embeddingss = OpenAIEmbeddings(openai_api_key='ssk-proj-nqRs9vvwyOvdfVqSnYXUT3BlbkFJtQbVBwMv3FROgN2JdtEK')

In [51]:
import os
PINECONE_API_KEY='8e9658d0-c1cb-48ce-a2ae-2b136b8bf0b0'
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [53]:
from langchain_pinecone import PineconeVectorStore

index_name = "naritwa"


docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

In [54]:
query = "What are women's rights?"
docs = docsearch.similarity_search(query)
print(docs[0].page_content)

(2) Every woman shall have the right to safe motherhood and reproductive 
health.  
(3) No woman shall be subjected to physical, mental, sexual, psychological or 
other  form of violence or exploitation on grounds of religion, social, cultural tradition, 
practice or on any other grounds. Such  act shall be punishable by law, and the victim 
shall have the right to obtain compensation in accordance with law.  
  (4) Women shal l have the right to participate in all bodies of the State on the


In [55]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [56]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [57]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [58]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [59]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

KeyboardInterrupt: 